# Pacotes que precisam ser instalados:
Comandos:
```shell
sudo pip install selenium    # Automatiza o navegador
sudo pip install bs4         # Html
sudo pip install html5lib    # 'parser' necessário para o bs4
sudo pip install textract    # PDF 
|--- sudo apt install swig          # dependencia do textract (instalacao)
|--- sudo apt install libpulse-dev  # dependencia do textract (pulseaudio)
```
<details>
    <summary> Comandos para serem instalados (Todos os pacotes de uma vez)</summary>
    <p><b>Instalação:</b><br><i>
        <b>sudo</b> apt install swig libpulse-dev <br>
        <b>sudo</b> pip install bs4 html5lib textract</i>
    </p>
    <p><b>Remoção:</b><br><i>
        <b>sudo</b> apt autoremove swig libpulse-dev <br>
        <b>sudo</b> pip uninstall bs4 html5lib textract</i>
    </p>
</details>

# Imports

In [2]:
from selenium import webdriver # automatiza nova guia do navegador em plano de fundo
from bs4 import BeautifulSoup  # necessário para trabalhar com o html
from selenium.webdriver.common.keys import Keys # Tecla ENTER

from os.path import abspath # diretorio
import textract # pdf
import re as regex # expressao regular para strings (limpeza de \t\n)

# import pickle # salvando em binario
# from os import listdir
from packages.SScraping import BasicXml # classe criada para gerar o XML

# URL's

In [3]:
url_login = "https://sigaa.unifei.edu.br/sigaa/verTelaLogin.do"
url_turma = "https://sigaa.unifei.edu.br/sigaa/portais/discente/turmas.jsf"
# url_grade = "https://sigaa.unifei.edu.br/sigaa//graduacao/curriculo/lista.jsf"
url_default = "https://sigaa.unifei.edu.br/sigaa/portais/discente/discente.jsf"
url_materia = "https://sigaa.unifei.edu.br/sigaa/geral/componente_curricular/busca_geral.jsf"

# Configurando o ambiente automatizado

In [4]:
pasta_projeto = abspath('') + '/'
pasta_webdriver = pasta_projeto + 'webdriver/chromedriver'
pasta_temp = pasta_projeto + '.temp_files/'
pasta_xml_files = pasta_projeto + 'xml_files/'

In [5]:
chrome_options = webdriver.ChromeOptions() # variavel de opcoes
# chrome_options.add_argument('--headless') # tire o comentário para ocultar a janela
chrome_options.binary_location = "/usr/bin/google-chrome" # caminho do executavel do chrome

# configura o chrome para salvar na pasta '.temp_files' do projeto,
chrome_options.add_experimental_option(
    "prefs",
    {
        'download.default_directory' : pasta_temp,
        'download.prompt_for_download' : False
    }
)

chrome_webdriver_path = pasta_webdriver # caminho do arquivo chromedriver

chrome_webdriver = webdriver.Chrome(executable_path=chrome_webdriver_path, options=chrome_options) 

# Fazendo login no _SIGAA_

In [6]:
name = '12638891665' #input("Digite o USUÁRIO do SIGAA: ")# Pede o login do sigaa
pwd  = 'Sigaa*0223'#input("Digite a SENHA do SIGAA: ")

In [7]:
try:
    chrome_webdriver.get(url_login) # acessando url

    user_name = chrome_webdriver.find_element_by_name("user.login") # encontra elemento
    user_name.clear() # limpa o campo
    user_name.send_keys(name) # envia o conteudo de name para o campo

    user_pwd = chrome_webdriver.find_element_by_name("user.senha")
    user_pwd.clear()
    user_pwd.send_keys(pwd)

    user_pwd.send_keys(Keys.ENTER) # tecla enter
    
except:
    print ('Elemento não encontrado na página -> login\nVerifique sua conexão!')

# Pegando as informações do Header do XML

In [8]:
try:
    curso = chrome_webdriver.find_element_by_css_selector('#agenda-docente > table > tbody > tr:nth-child(2) > td:nth-child(2)').text
    grade = '' # obtem no pdf
    matricula = chrome_webdriver.find_element_by_css_selector('#agenda-docente > table > tbody > tr:nth-child(1) > td:nth-of-type(2)').text
    anoperiodoinicial = chrome_webdriver.find_element_by_css_selector('#agenda-docente > table > tbody > tr:nth-child(6) > td:nth-child(2)').text
    anoperiodoatual = '' # obtem no pdf
except:
    print ('Elemento não encontrado na página -> login(main page)\nVerifique sua conexão!')

# Montando Historico Inicial (Nome,sigla,CH,turma)

In [83]:
def get_CH(nome):
    """
    Funcao usada para retornar a carga horaria da matéria.\n
    @param nome Sigla da materia que deseja-se obter a CH.\n
    @return (str) 'ch'
    """
    chrome_webdriver.get(url_materia)
    
    chrome_webdriver.find_element_by_name('formBusca:checkCodigo').click()

    box = chrome_webdriver.find_element_by_name('formBusca:j_id_jsp_433775776_860')
    box.clear()
    box.send_keys(nome)
    box.send_keys(Keys.ENTER) # shows 'listagem' 

    return chrome_webdriver.find_element_by_css_selector('#conteudo > table > tbody > tr > td:nth-child(4)').text

In [9]:
historico = {}
chrome_webdriver.get(url_turma)
soup = BeautifulSoup(chrome_webdriver.page_source, 'html5lib')
soup = soup.find('table', class_='listagem')

for i in soup.find_all('tr'):  # , class_='linhaPar'):
    # escape from tr' that are not importants to us
    if i.get('class') is None or i.get('class')[0] == 'destaque':
        continue
    itsigla = i.find('td').text
    itnome = itsigla[itsigla.find(' - ') + 3:]
    itsigla = itsigla[:itsigla.find(' -')]
    periodo = (i.find_previous('td', class_='periodo').text)
    itturma = i.find_next('td').find_next('td').text
    ch = (i.find_next('td').find_next('td').find_next('td')).text
    
    
    if (periodo not in historico):
        historico[periodo] = {
            itsigla: {
                'nome': itnome,
                'turma': itturma,
                'ch': ch[:-1]
            }
        }

    else:
        historico[periodo].update({
            itsigla: {
                'nome': itnome,
                'turma': itturma,
                'ch': ch[:-1]
            }
        })
    

In [10]:
historico

{'2019.1': {'EELI12': {'nome': 'ELETRÔNICA BÁSICA II',
   'turma': '01',
   'ch': '64'},
  'EELI14': {'nome': 'ELETRÔNICA DIGITAL', 'turma': '01', 'ch': '32'},
  'ECOI22': {'nome': 'INTELIGÊNCIA ARTIFICIAL', 'turma': '01', 'ch': '64'},
  'ECOI25': {'nome': 'INTERFACE HOMEM-COMPUTADOR', 'turma': '01', 'ch': '32'},
  'ECOI10': {'nome': 'LABORATÓRIO DE CIRCUITOS ELÉTRICOS I',
   'turma': '01',
   'ch': '16'},
  'EELI13': {'nome': 'LABORATÓRIO DE ELETRÔNICA BÁSICA II',
   'turma': '02',
   'ch': '32'},
  'EELI15': {'nome': 'LABORATÓRIO DE ELETRÔNICA DIGITAL',
   'turma': '02',
   'ch': '32'},
  'FISI06': {'nome': 'LABORATÓRIO FÍSICA B (ELETROMAGNETISMO, ÓTICA E FÍSICA MODERNA)',
   'turma': '03',
   'ch': '32'},
  'ECAI29': {'nome': 'MODELAGEM E ANÁLISE DE SISTEMAS DINÂMICOS',
   'turma': '01',
   'ch': '64'},
  'ECOI11': {'nome': 'PROJETO E ANÁLISE DE ALGORITMOS',
   'turma': '01',
   'ch': '64'},
  'ECOI13': {'nome': 'TEORIA DA COMPUTAÇÃO', 'turma': '01', 'ch': '48'}},
 '2018.2': {'EPRI0

# Adicionando informações ao histórico (notas,faltas,situacao)

In [11]:
chrome_webdriver.get(url_default) # acessa as notas pela func cmItemMouseup
js_script = "try{cmItemMouseUp('menu_form_menu_discente_j_id_jsp_275447739_49_menu',1);}catch(e){}" # script encontrado que gerou a página
chrome_webdriver.execute_script(js_script)
soup = BeautifulSoup(chrome_webdriver.page_source, 'html5lib')


for i in soup.find_all_next('tr', class_='linha'):
    sigla = i.find_next('td')
    resultado = sigla.find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
    faltas = resultado.find_next('td')
    situacao = faltas.find_next('td')
    
    sigla = sigla.text.replace(' ', '')
    sigla = regex.sub('[\n\t]', "", sigla)
    
    resultado = resultado.text.replace(' ', '')
    resultado = regex.sub('[\n\t]', "", resultado)
    
    faltas = faltas.text.replace(' ','')
    faltas = regex.sub('[\n\t]', "", faltas)
    
    situacao = situacao.text.replace(' ','')
    situacao = regex.sub('[\n\t]', "", situacao)
    
    periodo = regex.sub('\s', '', i.find_previous('caption').text)
    
    if (sigla in historico[periodo]): # considera apenas materias (exclui resultado dos blocos)        
        historico[periodo][sigla].update({
        'resultado': resultado,
        'faltas': faltas,
        'situacao': situacao
    })
    

# Obtendo grade e período atual do aluno

In [12]:
# * Obtendo o arquivo histórico.
# **Nota:**
# Têm parâmetros errados na obtenção do scraping, porém, funciona. Se funciona, tá top!

try:
    fileExist = open((pasta_temp + 'historico_{}.pdf'.format(matricula)), 'r') # verifica se existe na temp

except FileNotFoundError:
    chrome_webdriver.get(url_default)
    js_script = "try{cmItemMouseUp('menu_form_menu_discente_j_id_jsp_275447739_49_menu',4);}catch(e){}" # script encontrado que gerou a página
    chrome_webdriver.execute_script(js_script)

finally:
    # Raspando o PDF
    pt = textract.process(
        '{}historico_{}.pdf'.format(pasta_temp,matricula ), 
        method='pdftotext'
    )
    pt = pt.decode("UTF-8")
    pt = str(pt)
    pt = pt.split('\n')
    n = []

    for i in pt:
        if not i.strip():
            continue
        else:
            n.append(i)
    pt = '\n'.join(n)

    search = pt.find('Currículo:\n') + 11
    grade = ''.join([i for i in pt[search:pt.find('\n',search)] if i.isdigit()])
    search = pt.find('Período Letivo Atual:\n') + 22
    anoperiodoatual = pt[search:pt.find('\n',search)]

# Fechando o navegador

In [13]:
chrome_webdriver.quit()

# ''Funções'' para incrementar semestralmente

In [14]:
incrementyear = lambda s: ( str(int(s[:-2]) + 1) + '.1' )
incrementsem  = lambda s: ( s[:-1] + '2' )
aplus = lambda s: ( incrementyear(s) if s[-1] == '2' else incrementsem(s) ) # incrementa semestralmente

In [15]:
nano = anoperiodoinicial
for i in range(1,int(anoperiodoatual)):
    nano = aplus(nano) 
anoperiodoatual = nano # muda para que apareca o ano no estilo 20XX.Y ao invés de K

In [16]:
anoperiodoinicial

'2016.1'

# Gerando XML

In [17]:
to_xml = BasicXml()
to_xml.create_tag('HistoricoCurricular','')
to_xml.create_tag('Curso',curso)
to_xml.create_tag('Grade',grade)
to_xml.create_tag('Matricula',matricula)
to_xml.create_tag('AnoPeriodoInicial',anoperiodoinicial)
to_xml.create_tag('AnoPeriodoAtual',anoperiodoatual)
to_xml.create_tag('Semestre', '')

freq = lambda c,s : str(round((int(c)-int(s))/int(c)*100)) # usado para calcular a frequencia

semestre = anoperiodoinicial
while True: # Do ...while
    to_xml.create_tag('A{}'.format(semestre), '')
    for sigla in historico[semestre]:
        to_xml.create_tag('Disciplina', '')
        to_xml.create_tag('Nome', historico[semestre][sigla]['nome'])
        to_xml.create_tag('Sigla', sigla)
        to_xml.create_tag('Frequencia', freq(
            historico[semestre][sigla]['ch'],
            historico[semestre][sigla]['faltas']))
        to_xml.create_tag('Nota', historico[semestre][sigla]['resultado'])
        to_xml.create_tag('CH', historico[semestre][sigla]['ch'])
        to_xml.create_tag('Situacao', historico[semestre][sigla]['situacao'])
        to_xml.end_tag() # Fim disciplina
    to_xml.end_tag() # fim A`periodo`
    
    if (semestre == anoperiodoatual):
        break
    semestre = aplus(semestre)
to_xml.end_tag() # Semestre
to_xml.end_tag() # HistoricoCurricular

# Salvando o arquivo XML

In [18]:
with open(pasta_xml_files + 'historico_{}.xml'.format(matricula), 'w', encoding='utf-8') as file:
    file.write(to_xml.getXml())